In [1]:
from io import StringIO
from Bio import SeqIO
from src.config.Config import Config
from src.orchestrator.MainOrchestrator import MainOrchestrator

## Operativdad con Biopython 

Este proyecto utiliza Biopython para analizar secuencias de ADN en formato FASTA, calculando el contenido de GC y traduciendo cada secuencia a aminoácidos. El flujo de trabajo incluye la adquisición de un archivo FASTA, el cálculo del contenido de GC, la traducción de las secuencias (asumiendo que son codificantes) y la creación de un nuevo archivo FASTA de salida con las secuencias de aminoácidos, preservando los identificadores originales. En este jupyter notebook nos centraremos en ver los resultados y el uso del código implementado.

*Objetivos:*

* Calcular el contenido de GC de cada secuencia de ADN.
* Traducir las secuencias a aminoácidos.
* Guardar las secuencias de aminoácidos en un archivo FASTA de salida.


### Estructura del proyecto
```
+---------------------+          +---------------------+          +---------------------+
|   MainOrchestrator  |          | DownloaderOrchestrator |       | ReaderOrchestrator  |
+---------------------+          +---------------------+          +---------------------+
| - downloader        |<-------->| - download_files    |<------->| - read_sequences     |
| - gc_calculator     |          +---------------------+          +---------------------+
| - reader            |
+---------------------+               |
                                      |
                       +---------------------+        +---------------------+
                       | Downloader          |        | BaseFastaReader     |
                       +---------------------+        +---------------------+
                       | - download()        |        | - read()            |
                       +---------------------+        +---------------------+
                               |                               |
                               |                               |
                       +---------------------+        +---------------------+
                       | Validator           |        | FastaBatchReader    |
                       +---------------------+        +---------------------+
                       | - validate()        |        | - read_batch()      |
                       +---------------------+        +---------------------+

                                        +---------------------+          
                                        |   GCOrchestrator    |          
                                        +---------------------+          
                                        | - calculate_gc      |
                                        +---------------------+
                                                 |
                                                 |
                                        +---------------------+
                                        |   Utils             |
                                        +---------------------+
                                        | - calculate_gc()    |
                                        +---------------------+
```

In [2]:
FASTA_FILE="NM_001301717.fasta"
OUTPUT_FILE="AMINO_ACIDS.fasta"

### Lectura del fichero `.fasta`

Al ejecutar el proceso de carga de un archivo FASTA, el flujo de código comienza en la clase `MainOrchestrator`, que centraliza y coordina las operaciones principales. Cuando un archivo FASTA es cargado, se delega esta tarea a `ReaderOrchestrator`, que se encarga de gestionar la lectura del archivo y de organizar el proceso de extracción de datos.

- **MainOrchestrator**: Esta clase inicia el proceso, definiendo los componentes y organizando los pasos. A través de su instancia de `ReaderOrchestrator`, llama al método `read_sequences` para cargar el archivo FASTA.

- **ReaderOrchestrator**: Esta clase es responsable de la lógica específica de lectura. Su función `read_sequences` gestiona el acceso al archivo FASTA, utilizando instancias de `BaseFastaReader` o `FastaBatchReader` para procesar las secuencias según el formato y la configuración deseados.

- **BaseFastaReader** y **FastaBatchReader**: Estas clases leen las secuencias del archivo. `BaseFastaReader` puede encargarse de la lectura simple de secuencias, mientras que `FastaBatchReader` maneja la lectura por lotes si se requiere procesar grandes volúmenes de secuencias en paralelo o en grupos. Ambas clases implementan métodos como `read` o `read_batch`, que devuelven las secuencias en un formato accesible para el resto del programa.

A medida que `ReaderOrchestrator` lee las secuencias, estas se devuelven a `MainOrchestrator`, que puede delegar a otros módulos (como el `GCOrchestrator`) para calcular el contenido de GC.


In [3]:
orchestrator = MainOrchestrator(Config)
fastaId, content = orchestrator.execute_read_single(FASTA_FILE)
print(f"El identificador del fichero fasta es {fastaId} y su contenido \n es: {content}")

El identificador del fichero fasta es NM_001301717.2 y su contenido 
 es: CTCTAGATGAGTCAGTGGAGGGCGGGTGGAGCGTTGAACCGTGAAGAGTGTGGTTGGGCGTAAACGTGGACTTAAACTCAGGAGCTAAGGGGGAAACCAATGAAAAGCGTGCTGGTGGTGGCTCTCCTTGTCATTTTCCAGGTATGCCTGTGTCAAGATGAGGTCACGGACGATTACATCGGAGACAACACCACAGTGGACTACACTTTGTTCGAGTCTTTGTGCTCCAAGAAGGACGTGCGGAACTTTAAAGCCTGGTTCCTCCCTATCATGTACTCCATCATTTGTTTCGTGGGCCTACTGGGCAATGGGCTGGTCGTGTTGACCTATATCTATTTCAAGAGGCTCAAGACCATGACCGATACCTACCTGCTCAACCTGGCGGTGGCAGACATCCTCTTCCTCCTGACCCTTCCCTTCTGGGCCTACAGCGCGGCCAAGTCCTGGGTCTTCGGTGTCCACTTTTGCAAGCTCATCTTTGCCATCTACAAGATGAGCTTCTTCAGTGGCATGCTCCTACTTCTTTGCATCAGCATTGACCGCTACGTGGCCATCGTCCAGGCTGTCTCAGCTCACCGCCACCGTGCCCGCGTCCTTCTCATCAGCAAGCTGTCCTGTGTGGGCATCTGGATACTAGCCACAGTGCTCTCCATCCCAGAGCTCCTGTACAGTGACCTCCAGAGGAGCAGCAGTGAGCAAGCGATGCGATGCTCTCTCATCACAGAGCATGTGGAGGCCTTTATCACCATCCAGGTGGCCCAGATGGTGATCGGCTTTCTGGTCCCCCTGCTGGCCATGAGCTTCTGTTACCTTGTCATCATCCGCACCCTGCTCCAGGCACGCAACTTTGAGCGCAACAAGGCCATCAAGGTGATCATCGCTGTGGTCGTGGTCTTCATAGTCTTCCAGCTGCCCTACAATGGGGT

### Calcular el contenido de GC de cada secuencia

El cálculo del contenido de GC en el proyecto sigue un flujo bien definido, coordinado principalmente por la clase `GCOrchestrator`. Esta clase se encarga de realizar los cálculos de GC para las secuencias de ADN, ofreciendo una implementación modular y fácil de mantener.

Existen dos implementaciones para el cálculo del contenido de GC:
1. Una implementación propia desarrollada específicamente para el proyecto.
2. Una implementación que hace uso de funciones integradas de la biblioteca Biopython, utilizada para verificar la exactitud de la implementación propia.

#### Flujo del cálculo de GC

- **GCOrchestrator**: Esta clase orquesta el proceso de cálculo del contenido de GC. Su método principal, `calculate_gc`, recibe las secuencias de ADN procesadas por `ReaderOrchestrator` y delega el cálculo de GC a dos métodos distintos: uno que utiliza la implementación propia y otro que emplea Biopython. Al hacer esto, se asegura que ambas implementaciones se ejecuten en paralelo, permitiendo así comparar resultados y verificar la precisión de la implementación propia.

- **Utils**: Esta clase contiene la lógica de la implementación propia del cálculo de GC. En el método `calculate_gc`, se cuenta la cantidad de bases G y C en cada secuencia y se calcula el porcentaje de GC dividiendo esta cantidad por el total de bases. Esta implementación fue diseñada para ser independiente de cualquier biblioteca de terceros, lo que facilita su personalización y control.

- **Comparación de resultados**: `GCOrchestrator` compara los resultados de ambas implementaciones (propia y Biopython) para validar que la implementación propia esté funcionando correctamente. Esta comparación ayuda a asegurar la precisión de los cálculos y sirve como verificación adicional.

Este flujo modular permite una verificación constante de la exactitud de los cálculos, mientras se mantienen separados los componentes principales para facilitar el mantenimiento y las futuras mejoras.


In [4]:
orchestrator.calculate_and_compare_gc_content(FASTA_FILE)

Custom GC Content for NM_001301717.2: 54.77%
Biopython GC Content for NM_001301717.2: 54.77%
Ambos métodos producen resultados idénticos -> resultado biopython (54.769511638521216), resultado implementación (54.769511638521216)


### Traducir las secuencias a aminoácidos.

### Guardar las secuencias de aminoácidos en un archivo FASTA de salida.
El guardado de las secuencias de aminoácidos generadas se gestiona mediante un conjunto de clases que garantizan que las secuencias se almacenen en un archivo FASTA manteniendo los identificadores y descripciones originales. Este flujo modular permite manejar el guardado de las secuencias de forma eficiente y organizada.

### Flujo para el Guardado de Secuencias de Aminoácidos

- **MainOrchestrator**: Después de traducir las secuencias de ADN a aminoácidos, `MainOrchestrator` delega el proceso de guardado a `SaverOrchestrator`, que coordina el guardado en el formato adecuado.

- **SaverOrchestrator**: Esta clase orquesta el proceso de guardado de las secuencias de aminoácidos en un archivo FASTA. `SaverOrchestrator` recibe las secuencias traducidas y llama a `FastaSaver` para almacenar cada secuencia en el formato FASTA, preservando la estructura y la información original del archivo de entrada.

- **FastaSaver**: Esta clase contiene la lógica específica para escribir las secuencias de aminoácidos en un archivo FASTA. Su método `save_to_fasta` toma cada secuencia de aminoácidos, junto con su identificador y descripción, y las guarda en el archivo de salida. `FastaSaver` garantiza que las secuencias se guarden correctamente y respeta el formato FASTA, permitiendo una fácil reutilización de los datos en otras aplicaciones.

Este flujo modular asegura que las secuencias de aminoácidos se almacenen correctamente en el formato adecuado, facilitando la manipulación y almacenamiento de datos en bioinformática y garantizando la precisión de la información almacenada.


In [ ]:
orchestrator.write_proteins_to_fasta(protein_records, OUTPUT_FILE)